This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'ZKbWtzzgtp4AXiDa3CF7'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"
params = dict(api_key=API_KEY,start_date='2020-05-07',end_date='2020-05-08')
res = requests.get(url=url,params=params)

if res.status_code == 200:
    print('Your request was successful!')

Your request was successful!


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

res.json().keys()

dict_keys(['dataset_data'])

In the first level of the dictionary, 'dataset_data' is the only key. All information is stored inside this key.

In [5]:
res.json()['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

Above keys are in the second level of dictionary. The information needed for this exercise is stored in keys 'column_names' and 'data'. 

'data' consists of relavant information, and 'column_names' consist of corresponding labels. Lets take a look at them

In [6]:
colnames = res.json()['dataset_data']['column_names']
data = res.json()['dataset_data']['data'][0]
for col, val in zip(colnames,data):
    print(col,':', val)

Date : 2020-05-08
Open : 96.35
High : 97.35
Low : 95.15
Close : 95.3
Change : None
Traded Volume : 134071.0
Turnover : 12869764.45
Last Price of the Day : None
Daily Traded Units : None
Daily Turnover : None


## These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [7]:
# url request
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"

# Request parameters defining api_key, start and end dates to access data only for the year 2017.
params = dict(api_key=API_KEY,start_date='2017-01-01',end_date='2017-12-31')
# Send request
res = requests.get(url=url,params=params)

# If the request status code is 200, that means we successfully received response for the request 
if res.status_code == 200:
    print('Your request was successful!')

Your request was successful!


2. Convert the returned JSON object into a Python dictionary.

In [8]:
data = res.json()
print('Data Type:', type(data))

Data Type: <class 'dict'>


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
from collections import defaultdict

# Store column names in a variable.
colnames = data['dataset_data']['column_names']

# Defining an empty dictionary with list as the default storage datatype 
daily_data = defaultdict(list)

# Fill the empty dictionary with data from each day in the year 2017, with proper key names
for obs in data['dataset_data']['data']:
    for key, val in zip(colnames,obs):
        daily_data[key].append(val)

print('Lowest opening price in year 2017: $%.2f' %(min([val for val in daily_data['Open'] if val is not None])))
print('Highest opening price in year 2017: $%.2f' %(max([val for val in daily_data['Open'] if val is not None])))

Lowest opening price in year 2017: $34.00
Highest opening price in year 2017: $53.11


4. What was the largest change in any one day (based on High and Low price)?

In [10]:
# Here change is defined as the absolute difference between High and Low prices of a particular day.

# Initialize an empty list.
change = []

# Append the empty list with absolute difference between High and Low prices for each day.
for high, low in zip(daily_data['High'],daily_data['Low']):
    if None not in [high,low]:
        change.append(abs(high-low))
        

print('Largest change in any one day (based on High and Low price): $%.2f' %(max(change)))

Largest change in any one day (based on High and Low price): $2.81


5. What was the largest change between any two days (based on Closing Price)?

In [11]:
# Here change is defined as the absolute difference between the closing prices of two consecutive days.

# Store closing prices in a variable.
close_price = daily_data['Close']

# Initialize a list with first element as 0 (this is only for convenience of indexing).
change = [0]

# Append the list with absolute difference between the closing prices of two consecutive days.
for i in range(1,len(close_price)):
    change.append(abs(close_price[i] - close_price[i-1]))
    
print('Largest change between any two days (based on Closing Price): $%.2f' %(max(change)))

Largest change between any two days (based on Closing Price): $2.56


6. What was the average daily trading volume during this year?

In [12]:
# Store daily trading volume as a list
tv = daily_data['Traded Volume']

# Compute and display its mean
mean_tv = sum(tv)/len(tv)
print('Average daily trading volume: $%.2f' %(mean_tv))

Average daily trading volume: $89124.34


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [13]:
# median function to compute median.
def median(array):
    array = sorted(array)
    # if odd number of values in array
    if len(array) % 2 == 1:
        idx = int(len(array)/2)
        median =  array[idx]
    # if even number of values in array
    else:
        idx = int(len(array)/2)
        median = (array[idx] + array[idx-1])/2
    return median

print('Median trading volume during year 2017: $%.2f' %(median(tv)))

Median trading volume during year 2017: $76286.00
